In [1]:
import os
import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "quanta.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
from apps.users.models import User

import csv
import os

In [3]:
import time
from django.db import transaction

In [4]:
# utils/data_migration.py (continued)
from decimal import Decimal
from django.utils import timezone
from phonenumber_field.phonenumber import PhoneNumber
from apps.orders.models import (
    Customer, Category, Item, Supplier, Batch, Stock,
    Facturation, FacturationStock, Transaction,
    FacturationPayment, Organization, OrganizationUser as NewOrganizationUser
)

In [5]:
# Create an organization with get_or_create
organization, created = Organization.objects.get_or_create(
    name="Agri distribution",
    defaults={
        'slug': 'main-hospital',
        'contact_email': 'contact@mainhospital.com',
        'credential': 'HOSP12345',
        'code': 'MH001',
        'sub_name': 'Main Branch',
        'country': 'Cameroon',
        'city': 'Douala',
        'street_address': '123 Main Street',
        'contact_number': '+237672120240',
        'short_name': 'MH',
        'tax_rate': Decimal('18.50'),
        'is_online': True,
        'timezone': 'Africa/Douala',
        'contribuable': '123456789',
        'created': timezone.now(),
        'modified': timezone.now(),
    }
)

if created:
    print(f"Created new organization: {organization.name}")
else:
    print(f"Organization already exists: {organization.name}")

Created new organization: Agri distribution


In [8]:
organization_id="69393ba2-3fbd-4ca8-a2d8-2e75ff14093d"

In [9]:
Organization.objects.filter(id=organization.id).update(id=organization_id)

1

In [14]:


def import_organization_data(export_dir='./demo/data', target_org_id=None):
    """
    Import data from CSV files to new models for a specific organization.
    
    Args:
        export_dir: Directory containing CSV files
        target_org_id: ID of the organization to import data for (required)
    
    Returns:
        tuple: (success: bool, message: str)
    """
    if not target_org_id:
        return False, "Error: target_org_id is required"
    
    try:
        with transaction.atomic():
            print(f"Starting data import for organization ID: {target_org_id}...")
            
           
            org_map = {target_org_id: target_org_id}
            
            # Import in correct order to maintain foreign key relationships
            user_map = _import_users(export_dir)
            ou_map = _import_organization_users(export_dir, org_map, user_map, target_org_id)
            customer_map = _import_customers(export_dir, org_map, target_org_id)
            supplier_map = _import_suppliers(export_dir, org_map, target_org_id)
            category_map = _import_categories(export_dir, org_map, target_org_id)
            item_map = _import_items(export_dir, org_map, category_map, target_org_id)
            batch_map = _import_batches(export_dir, org_map, item_map, supplier_map, ou_map, target_org_id)
            
            # Import facturations first
            facturation_map = _import_facturations(export_dir, org_map, customer_map, ou_map, target_org_id)
            
            # Now create Stock records from FacturationBatch data
            stock_map = _create_stocks_from_facturation_batches(
                export_dir, org_map, batch_map, ou_map, target_org_id
            )
            
            # Import FacturationStock
            _import_facturation_stocks(export_dir, org_map, stock_map, facturation_map, ou_map, target_org_id)
            
            # Import remaining models
            _import_transactions(export_dir, org_map, ou_map, target_org_id)
            _import_facturation_payments(export_dir, org_map, facturation_map, ou_map, target_org_id)
            
        return True, f'Data import completed successfully for organization ID: {target_org_id}!'
        
    except Exception as e:
        return False, f"Import failed: {str(e)}"
def _import_users(export_dir):
    """Import users from export"""
    filepath = os.path.join(export_dir, 'users.csv')
    user_map = {}
    
    if not os.path.exists(filepath):
        print(f"Warning: Users file not found: {filepath}")
        return user_map
    
    with open(filepath, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        users_to_create = []
        
        for row in reader:
            try:
                # Check if user already exists by email
                existing_user = User.objects.filter(email=row['email']).first()
                if existing_user:
                    user_map[str(row['id'])] = existing_user
                    continue
                
                # Create user
                user = User(
                    id=str(row['id']),
                    email=row['email'] or f"user{row['id']}@example.com",
                    username=row['username'] or f"user{row['id']}",
                    is_patient=row['is_patient'].lower() == 'true' if row.get('is_patient') else True,
                    is_staff=row['is_staff'].lower() == 'true' if row.get('is_staff') else False,
                    is_active=row['is_active'].lower() == 'true' if row.get('is_active') else True,
                    is_superuser=row['is_superuser'].lower() == 'true' if row.get('is_superuser') else False,
                    date_joined=_parse_datetime(row.get('date_joined')),
                    last_login=_parse_datetime(row.get('last_login')),
                )
                
                # Handle phone number
                if row.get('phone_number'):
                    try:
                        user.phone_number = PhoneNumber.from_string(row['phone_number'])
                    except:
                        user.phone_number = None
                
                users_to_create.append(user)
                user_map[str(row['id'])] = user
                
            except Exception as e:
                print(f"Error importing user {row.get('id')}: {e}")
                continue
        
        if users_to_create:
            # Set default password for all users
            for user in users_to_create:
                if not user.has_usable_password():
                    user.set_password('DataViteAI')
            
            User.objects.bulk_create(users_to_create)
            print(f"Imported {len(users_to_create)} users")
    
    return user_map

def _import_organization_users(export_dir, org_map, user_map, target_org_id):
    """Import organization users for target organization"""
    filepath = os.path.join(export_dir, 'organization_users.csv')
    ou_map = {}
    
    if not os.path.exists(filepath):
        print(f"Warning: Organization users file not found: {filepath}")
        return ou_map
    
    with open(filepath, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        ous_to_create = []
        
        for row in reader:
            try:
                org_id = str(row['organization_id'])
                user_id = str(row['user_id'])
                
                # Only import for target organization
                if org_id != target_org_id:
                    continue
                
                if org_id not in org_map or user_id not in user_map:
                    print(f"Skipping organization user {row['id']}: org={org_id}, user={user_id}")
                    continue
                
                # Check if already exists
                existing_ou = NewOrganizationUser.objects.filter(
                    organization_id=org_map[org_id],
                    user=user_map[user_id],
                    is_active=True,
                    can_edit_price=True,
                    can_print_transaction=True,
                    can_print_bill=True,
                ).first()
                
                if existing_ou:
                    ou_map[str(row['id'])] = existing_ou
                    continue
                
                ou = NewOrganizationUser(
                    id=str(row['id']),
                    organization_id=org_map[org_id],
                    user=user_map[user_id],
                    created=_parse_datetime(row.get('created')),
                    modified=_parse_datetime(row.get('modified'))
                )
                ous_to_create.append(ou)
                ou_map[str(row['id'])] = ou
            except Exception as e:
                print(f"Error importing organization user {row.get('id')}: {e}")
                continue
        
        if ous_to_create:
            NewOrganizationUser.objects.bulk_create(ous_to_create)
            print(f"Imported {len(ous_to_create)} organization users")
    
    return ou_map

def _import_customers(export_dir, org_map, target_org_id):
    """Import customers for target organization"""
    filepath = os.path.join(export_dir, 'customers.csv')
    customer_map = {}
    
    if not os.path.exists(filepath):
        print(f"Warning: Customers file not found: {filepath}")
        return customer_map
    
    with open(filepath, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        customers_to_create = []
        
        for row in reader:
            try:
                org_id = str(row['organization_id'])
                
                # Only import for target organization
                if org_id != target_org_id:
                    continue
                
                if org_id not in org_map:
                    print(f"Skipping customer {row['id']}: org {org_id} not found")
                    continue
                
                customer = Customer(
                    id=str(row['id']),
                    organization_id=org_map[org_id],
                    name=row['name'],
                    phone_number=row['phone_number'] or None,
                    created=_parse_datetime(row.get('created')),
                    modified=_parse_datetime(row.get('modified'))
                )
                customers_to_create.append(customer)
                customer_map[str(row['id'])] = customer
            except Exception as e:
                print(f"Error importing customer {row.get('id')}: {e}")
                continue
        
        if customers_to_create:
            Customer.objects.bulk_create(customers_to_create)
            print(f"Imported {len(customers_to_create)} customers")
    
    return customer_map

def _import_suppliers(export_dir, org_map, target_org_id):
    """Import suppliers for target organization"""
    filepath = os.path.join(export_dir, 'suppliers.csv')
    supplier_map = {}
    
    if not os.path.exists(filepath):
        print(f"Warning: Suppliers file not found: {filepath}")
        return supplier_map
    
    with open(filepath, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        suppliers_to_create = []
        
        for row in reader:
            try:
                org_id = str(row['organization_id'])
                
                # Only import for target organization
                if org_id != target_org_id:
                    continue
                
                if org_id not in org_map:
                    print(f"Skipping supplier {row['id']}: org {org_id} not found")
                    continue
                
                supplier = Supplier(
                    id=str(row['id']),
                    organization_id=org_map[org_id],
                    name=row['name'],
                    is_active=row['is_active'].lower() == 'true' if row['is_active'] else True,
                    created=_parse_datetime(row.get('created')),
                    modified=_parse_datetime(row.get('modified'))
                )
                suppliers_to_create.append(supplier)
                supplier_map[str(row['id'])] = supplier
            except Exception as e:
                print(f"Error importing supplier {row.get('id')}: {e}")
                continue
        
        if suppliers_to_create:
            Supplier.objects.bulk_create(suppliers_to_create)
            print(f"Imported {len(suppliers_to_create)} suppliers")
    
    return supplier_map

def _import_categories(export_dir, org_map, target_org_id):
    """Import categories for target organization"""
    filepath = os.path.join(export_dir, 'categories.csv')
    category_map = {}
    
    if not os.path.exists(filepath):
        print(f"Warning: Categories file not found: {filepath}")
        return category_map
    
    with open(filepath, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        categories_to_create = []
        
        for row in reader:
            try:
                org_id = str(row['organization_id'])
                
                # Only import for target organization
                if org_id != target_org_id:
                    continue
                
                if org_id not in org_map:
                    print(f"Skipping category {row['id']}: org {org_id} not found")
                    continue
                
                category = Category(
                    id=str(row['id']),
                    organization_id=org_map[org_id],
                    name=row['name'],
                    is_active=row['is_active'].lower() == 'true' if row.get('is_active') else True,
                    created=_parse_datetime(row.get('created')),
                    modified=_parse_datetime(row.get('modified'))
                )
                categories_to_create.append(category)
                category_map[str(row['id'])] = category
            except Exception as e:
                print(f"Error importing category {row.get('id')}: {e}")
                continue
        
        if categories_to_create:
            Category.objects.bulk_create(categories_to_create)
            print(f"Imported {len(categories_to_create)} categories")
    
    return category_map

def _import_items(export_dir, org_map, category_map, target_org_id):
    """Import items for target organization"""
    filepath = os.path.join(export_dir, 'items.csv')
    item_map = {}
    
    if not os.path.exists(filepath):
        print(f"Warning: Items file not found: {filepath}")
        return item_map
    
    with open(filepath, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        items_to_create = []
        
        for row in reader:
            try:
                org_id = str(row['organization_id'])
                category_id = str(row['category_id'])
                
                # Only import for target organization
                if org_id != target_org_id:
                    continue
                
                if org_id not in org_map or category_id not in category_map:
                    print(f"Skipping item {row['id']}: org={org_id}, category={category_id}")
                    continue
                
                item = Item(
                    id=str(row['id']),
                    organization_id=org_map[org_id],
                    category=category_map[category_id],
                    name=row['name'],
                    alert_quantity=int(row['alert_quantity']) if row['alert_quantity'] else 1,
                    is_active=row['is_active'].lower() == 'true' if row.get('is_active') else True,
                    created=_parse_datetime(row.get('created')),
                    modified=_parse_datetime(row.get('modified'))
                )
                items_to_create.append(item)
                item_map[str(row['id'])] = item
            except Exception as e:
                print(f"Error importing item {row.get('id')}: {e}")
                continue
        
        if items_to_create:
            Item.objects.bulk_create(items_to_create)
            print(f"Imported {len(items_to_create)} items")
    
    return item_map

def _import_batches(export_dir, org_map, item_map, supplier_map, ou_map, target_org_id):
    """Import batches for target organization"""
    filepath = os.path.join(export_dir, 'batches.csv')
    batch_map = {}
    
    if not os.path.exists(filepath):
        print(f"Warning: Batches file not found: {filepath}")
        return batch_map
    
    with open(filepath, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        batches_to_create = []
        
        for row in reader:
            try:
                org_id = str(row['organization_id'])
                item_id = str(row['item_id'])
                supplier_id = str(row['supplier_id'])
                last_maintainer_id = str(row['last_maintainer_id'])
                
                # Only import for target organization
                if org_id != target_org_id:
                    continue
                
                if (org_id not in org_map or item_id not in item_map or 
                    supplier_id not in supplier_map or last_maintainer_id not in ou_map):
                    print(f"Skipping batch {row['id']}: missing foreign keys")
                    continue
                
                batch = Batch(
                    id=str(row['id']),
                    organization_id=org_map[org_id],
                    item=item_map[item_id],
                    batch_number=row['batch_number'] or f"BATCH-{row['id']}",
                    supplier=supplier_map[supplier_id],
                    received_date=_parse_date(row['received_date']),
                    expiration_date=_parse_date(row['expiration_date']),
                    purchase_price=Decimal(row['purchase_price']) if row['purchase_price'] else Decimal('0'),
                    facturation_price=Decimal(row['facturation_price']) if row['facturation_price'] else Decimal('0'),
                    quantity=int(float(row['quantity'])) if row['quantity'] else 0,
                    last_checked=_parse_datetime(row.get('last_checked')),
                    last_maintainer=ou_map[last_maintainer_id],
                    is_active=row['is_active'].lower() == 'true' if row.get('is_active') else True,
                    created=_parse_datetime(row.get('created')),
                    modified=_parse_datetime(row.get('modified'))
                )
                batches_to_create.append(batch)
                batch_map[str(row['id'])] = batch
            except Exception as e:
                print(f"Error importing batch {row.get('id')}: {e}")
                continue
        
        if batches_to_create:
            Batch.objects.bulk_create(batches_to_create)
            print(f"Imported {len(batches_to_create)} batches")
    
    return batch_map

def _import_facturations(export_dir, org_map, customer_map, ou_map, target_org_id):
    """Import facturations for target organization"""
    filepath = os.path.join(export_dir, 'facturations.csv')
    facturation_map = {}
    
    if not os.path.exists(filepath):
        print(f"Warning: Facturations file not found: {filepath}")
        return facturation_map
    
    with open(filepath, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        facturations_to_create = []
        
        for row in reader:
            try:
                org_id = str(row['organization_id'])
                customer_id = str(row['customer_id'])
                org_user_id = str(row['organization_user_id'])
                
                # Only import for target organization
                if org_id != target_org_id:
                    continue
                
                if (org_id not in org_map or customer_id not in customer_map or 
                    org_user_id not in ou_map):
                    print(f"Skipping facturation {row['id']}: missing foreign keys")
                    continue
                
                facturation = Facturation(
                    id=str(row['id']),
                    organization_id=org_map[org_id],
                    customer=customer_map[customer_id],
                    custom_customer=row['custom_customer'] or None,
                    organization_user=ou_map[org_user_id],
                    bill_number=row['bill_number'] or f"FACT-{row['id']}",
                    is_delivered=row['is_delivered'].lower() == 'true' if row.get('is_delivered') else True,
                    is_proforma=row['is_proforma'].lower() == 'true' if row.get('is_proforma') else False,
                    placed_at=_parse_datetime(row['placed_at']),
                    created=_parse_datetime(row.get('created')),
                    modified=_parse_datetime(row.get('modified'))
                )
                facturations_to_create.append(facturation)
                facturation_map[str(row['id'])] = facturation
            except Exception as e:
                print(f"Error importing facturation {row.get('id')}: {e}")
                continue
        
        if facturations_to_create:
            Facturation.objects.bulk_create(facturations_to_create)
            print(f"Imported {len(facturations_to_create)} facturations")
    
    return facturation_map

def _create_stocks_from_facturation_batches(export_dir, org_map, batch_map, ou_map, target_org_id):
    """Create Stock records from FacturationBatch data for target organization."""
    filepath = os.path.join(export_dir, 'facturation_batches.csv')
    stock_map = {}
    
    if not os.path.exists(filepath):
        print(f"Warning: Facturation batches file not found: {filepath}")
        return stock_map
    
    # First, read all facturation batches to calculate required stock quantities
    batch_stock_required = {}
    
    with open(filepath, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                org_id = str(row['organization_id'])
                
                # Only process for target organization
                if org_id != target_org_id:
                    continue
                
                batch_id = str(row['batch_id'])
                quantity = int(float(row['quantity'])) if row['quantity'] else 0
                
                if batch_id not in batch_stock_required:
                    batch_stock_required[batch_id] = 0
                batch_stock_required[batch_id] += quantity
            except:
                continue
    
    # Create Stock records
    stocks_to_create = []
    stock_counter = 1
    
    for batch_id, total_quantity in batch_stock_required.items():
        try:
            if batch_id not in batch_map:
                print(f"Skipping stock for batch {batch_id}: batch not found")
                continue
            
            batch = batch_map[batch_id]
            org_id = batch.organization_id
            
            # Get a valid organization user for this stock
            org_user = None
            for ou in ou_map.values():
                if ou.organization_id == org_id:
                    org_user = ou
                    break
            
            if not org_user:
                print(f"No organization user found for org {org_id}, skipping stock")
                continue
            
            # Create one stock record per batch with the total required quantity
            stock = Stock(
                id=stock_counter,
                organization_id=org_map[org_id],
                organization_user=org_user,
                batch=batch,
                quantity=total_quantity
            )
            stocks_to_create.append(stock)
            stock_map[batch_id] = stock
            stock_counter += 1
        except Exception as e:
            print(f"Error creating stock for batch {batch_id}: {e}")
            continue
    
    if stocks_to_create:
        Stock.objects.bulk_create(stocks_to_create)
        print(f"Created {len(stocks_to_create)} stock records")
    
    return stock_map

def _import_facturation_stocks(export_dir, org_map, stock_map, facturation_map, ou_map, target_org_id):
    """Import FacturationStock from FacturationBatch data for target organization"""
    filepath = os.path.join(export_dir, 'facturation_batches.csv')
    
    if not os.path.exists(filepath):
        print(f"Warning: Facturation batches file not found: {filepath}")
        return
    
    with open(filepath, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        facturation_stocks_to_create = []
        
        for row in reader:
            try:
                org_id = str(row['organization_id'])
                facturation_id = str(row['facturation_id'])
                batch_id = str(row['batch_id'])
                org_user_id = str(row['organization_user_id'])
                
                # Only import for target organization
                if org_id != target_org_id:
                    continue
                
                if (facturation_id not in facturation_map or batch_id not in stock_map or
                    org_id not in org_map or org_user_id not in ou_map):
                    print(f"Skipping facturation stock {row['id']}: missing foreign keys")
                    continue
                
                facturation_stock = FacturationStock(
                    id=str(row['id']),  # Preserve original FacturationBatch ID
                    organization_id=org_map[org_id],
                    stock=stock_map[batch_id],
                    organization_user=ou_map[org_user_id],
                    facturation=facturation_map[facturation_id],
                    unit_price=Decimal(row['unit_price']) if row['unit_price'] else Decimal('0'),
                    quantity=int(float(row['quantity'])) if row['quantity'] else 0,
                    is_delivered=row['is_delivered'].lower() == 'true' if row.get('is_delivered') else False,
                    created=_parse_datetime(row.get('created')),
                    modified=_parse_datetime(row.get('modified'))
                )
                facturation_stocks_to_create.append(facturation_stock)
            except Exception as e:
                print(f"Error importing facturation stock {row.get('id')}: {e}")
                continue
        
        if facturation_stocks_to_create:
            FacturationStock.objects.bulk_create(facturation_stocks_to_create)
            print(f"Imported {len(facturation_stocks_to_create)} facturation stocks")

def _import_transactions(export_dir, org_map, ou_map, target_org_id):
    """Import transactions for target organization"""
    filepath = os.path.join(export_dir, 'transactions.csv')
    
    if not os.path.exists(filepath):
        print(f"Warning: Transactions file not found: {filepath}")
        return
    
    with open(filepath, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        transactions_to_create = []
        
        for row in reader:
            try:
                org_id = str(row['organization_id'])
                org_user_id = str(row['organization_user_id'])
                
                # Only import for target organization
                if org_id != target_org_id:
                    continue
                
                if org_id not in org_map or org_user_id not in ou_map:
                    print(f"Skipping transaction {row['id']}: missing foreign keys")
                    continue
                
                transaction = Transaction(
                    id=str(row['id']),
                    organization_id=org_map[org_id],
                    organization_user=ou_map[org_user_id],
                    transaction_broker=row['transaction_broker'],
                    transaction_type=row['transaction_type'],
                    amount=Decimal(row['amount']) if row['amount'] else Decimal('0'),
                    participant=row['participant'] or '',
                    reason=row['reason'] or '',
                    created=_parse_datetime(row.get('created')),
                    modified=_parse_datetime(row.get('modified'))
                )
                transactions_to_create.append(transaction)
            except Exception as e:
                print(f"Error importing transaction {row.get('id')}: {e}")
                continue
        
        if transactions_to_create:
            Transaction.objects.bulk_create(transactions_to_create)
            print(f"Imported {len(transactions_to_create)} transactions")

def _import_facturation_payments(export_dir, org_map, facturation_map, ou_map, target_org_id):
    """Import facturation payments for target organization"""
    filepath = os.path.join(export_dir, 'facturation_payments.csv')
    
    if not os.path.exists(filepath):
        print(f"Warning: Facturation payments file not found: {filepath}")
        return
    
    with open(filepath, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        payments_to_create = []
        
        for row in reader:
            try:
                org_id = str(row['organization_id'])
                facturation_id = str(row['facturation_id'])
                org_user_id = str(row['organization_user_id'])
                
                # Only import for target organization
                if org_id != target_org_id:
                    continue
                
                if (org_id not in org_map or facturation_id not in facturation_map or 
                    org_user_id not in ou_map):
                    print(f"Skipping facturation payment {row['id']}: missing foreign keys")
                    continue
                
                payment = FacturationPayment(
                    id=str(row['id']),
                    organization_id=org_map[org_id],
                    facturation=facturation_map[facturation_id],
                    organization_user=ou_map[org_user_id],
                    transaction_broker=row['transaction_broker'],
                    amount=Decimal(row['amount']) if row['amount'] else Decimal('0'),
                    created=_parse_datetime(row.get('created')),
                    modified=_parse_datetime(row.get('modified'))
                )
                payments_to_create.append(payment)
            except Exception as e:
                print(f"Error importing facturation payment {row.get('id')}: {e}")
                continue
        
        if payments_to_create:
            FacturationPayment.objects.bulk_create(payments_to_create)
            print(f"Imported {len(payments_to_create)} facturation payments")

def _parse_datetime(value):
    """Parse datetime string safely"""
    if not value:
        return timezone.now()
    try:
        return timezone.datetime.fromisoformat(value.replace('Z', '+00:00'))
    except (ValueError, AttributeError):
        return timezone.now()

def _parse_date(value):
    """Parse date string safely"""
    if not value:
        return timezone.now().date()
    try:
        return timezone.datetime.fromisoformat(value.replace('Z', '+00:00')).date()
    except (ValueError, AttributeError):
        return timezone.now().date()

In [15]:
# Then import
import_success, import_message = import_organization_data(
        export_dir='./demo/data',
        target_org_id=organization_id
    )

Starting data import for organization ID: 69393ba2-3fbd-4ca8-a2d8-2e75ff14093d...
